In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#            Machine Learning y Deep Learning
# ----------------------------------------------------------
#         Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
#        Redes neuronales LSTM con regularizacion
# ==========================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
# ---------------------- Cargando datos ----------------------------------------
# Cargar el conjunto de datos:
url =  "https://raw.githubusercontent.com/rogon666/UMSA/main/AIMLDL/Datos/BTCUSD.csv"
# Cargar los datos en un DataFrame
data= pd.read_csv(url)
# Mostrar las primeras filas del DataFrame
print(data.head())
# Preparar el objetivo (y)
y = data['Close'].values.reshape(-1, 1)

# Crear la matriz de características X como rezagos de y
def create_lagged_features(y, lag=1):
    X = np.zeros((len(y) - lag, lag))
    for i in range(lag):
        X[:, i] = y[(lag - i - 1):(len(y) - i - 1), 0]
    y_lagged = y[lag:]
    return X, y_lagged

# Usar rezagos para las características
lag = 60
X, y_lagged = create_lagged_features(y, lag)

# Normalizar los datos
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X = scaler_X.fit_transform(X)
y_lagged = scaler_y.fit_transform(y_lagged)



# Reshape para que sea compatible con LSTM (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Partición de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y_lagged,
                                                    test_size=0.15,
                                                    shuffle=False)


         Date          Open          High           Low         Close  \
0  2019-08-21  10764.572266  10798.729492   9962.721680  10138.049805   
1  2019-08-22  10142.521484  10232.996094   9831.462891  10131.055664   
2  2019-08-23  10136.309570  10442.443359  10078.192383  10407.964844   
3  2019-08-24  10407.644531  10418.020508   9982.296875  10159.960938   
4  2019-08-25  10160.737305  10304.622070  10008.789063  10138.517578   

      Adj Close       Volume  
0  10138.049805  19473084768  
1  10131.055664  17097508856  
2  10407.964844  15627023886  
3  10159.960938  15451030650  
4  10138.517578  14153856610  


In [ ]:
# Definir el modelo LSTM con Dropout
model = Sequential()
model.add(LSTM(units=50,
               return_sequences=True,
               input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Apagar el 20% de las neuronas de esta capa
model.add(LSTM(units=50))
model.add(Dropout(0.2))  # Apagar el 20% de las neuronas de esta capa
model.add(Dense(1))

# Compilación del modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=10)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


47/47 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - loss: 0.0472
Epoch 2/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - loss: 0.0143
Epoch 3/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0126
Epoch 4/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0080
Epoch 5/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.0072
Epoch 6/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - loss: 0.0056
Epoch 7/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0057
Epoch 8/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0047
Epoch 9/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - loss: 0.0049
Epoch 10/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - loss: 0.0048
Epoch 11/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 0.0059
Epoch 12/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0057
Epoch 13/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.0052
Epoch 14/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - loss: 0.0036
Epoch 15/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0031
E

In [ ]:
# Modelo LSTM con Regularización L2
model = Sequential()
model.add(LSTM(units=50,
               return_sequences=True,
               input_shape=(X_train.shape[1], 1),
               kernel_regularizer=l2(0.01)))  # Agregar regularización L2
model.add(LSTM(units=50, kernel_regularizer=l2(0.01)))  # Agregar regularización L2
model.add(Dense(1))

# Compilación del modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=10)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


47/47 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - loss: 0.7436
Epoch 2/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - loss: 0.2897
Epoch 3/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 7s 90ms/step - loss: 0.1081
Epoch 4/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - loss: 0.0403
Epoch 5/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - loss: 0.0181
Epoch 6/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.0130
Epoch 7/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.0075
Epoch 8/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - loss: 0.0060
Epoch 9/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.0057
Epoch 10/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.0045
Epoch 11/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - loss: 0.0050
Epoch 12/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 85ms/step - loss: 0.0042
Epoch 13/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - loss: 0.0034
Epoch 14/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 0.0037
Epoch 15/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 0.0031
E

In [ ]:
# Definir el modelo LSTM con Dropout y Regularización L2
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1),
               kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(1))

# Compilación del modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=10)
